# 010: The Hopf Oscillator Model

A deep dive into the Hopf oscillator for modeling mesoscopic neural population dynamics and bifurcation behavior.

## Learning Objectives

By the end of this tutorial, you will be able to:

- Understand the mathematical formulation of the Hopf oscillator
- Explain the role of each parameter (a, w, beta) in shaping dynamics
- Generate and interpret bifurcation diagrams
- Build a multi-region brain network with Hopf nodes and diffusive coupling
- Compute functional connectivity from simulated neural activity

## Prerequisites

- **Tutorial 000**: Getting Started with BrainMass
- Basic understanding of differential equations
- Familiarity with complex numbers (helpful but not required)

## Background / Theory

### The Hopf Bifurcation

The **Hopf oscillator** is a canonical model describing dynamics near a Hopf bifurcation - a mathematical transition where a stable fixed point becomes unstable and gives rise to a stable limit cycle (oscillation).

In neuroscience, this captures how brain regions transition between:
- **Quiescent state** (a < 0): Activity decays to zero
- **Critical point** (a = 0): Bifurcation threshold
- **Oscillatory state** (a > 0): Persistent rhythmic activity

### Mathematical Formulation

The supercritical Hopf oscillator in complex form:

$$
\frac{dz}{dt} = (a + i\omega) z - \beta |z|^2 z + I_{\text{ext}}(t)
$$

where $z = x + iy$ and $|z|^2 = x^2 + y^2$.

In real coordinates $(x, y)$:

$$
\begin{aligned}
\dot{x} &= (a - \beta r)x - \omega y + I_x(t) \\
\dot{y} &= (a - \beta r)y + \omega x + I_y(t) \\
r &= x^2 + y^2
\end{aligned}
$$

### Parameters

| Parameter | Symbol | Role | Typical Range |
|-----------|--------|------|---------------|
| Bifurcation | $a$ | Controls proximity to oscillation onset | -1 to 1 |
| Frequency | $\omega$ | Angular frequency of oscillation | 0.1 to 1.0 |
| Saturation | $\beta$ | Limits oscillation amplitude | > 0 |

For $a > 0$, the limit cycle has radius $\approx \sqrt{a/\beta}$.

## Implementation

### Step 1: Setup and Imports

In [ ]:
import jax
import brainmass
import brainstate
import braintools
import brainunit as u
import matplotlib.pyplot as plt
import numpy as np

# Set simulation time step
brainstate.environ.set(dt=0.1 * u.ms)

### Step 2: Single Node Simulation

Let's create a single Hopf oscillator with Ornstein-Uhlenbeck (OU) noise to simulate realistic neural fluctuations:

In [ ]:
# Create a single Hopf node with noise
node = brainmass.HopfStep(
    1,                # single node
    a=0.25,           # above bifurcation -> oscillating
    w=0.2,            # angular frequency
    beta=1.0,         # amplitude saturation
    noise_x=brainmass.OUProcess(1, sigma=0.01),
    noise_y=brainmass.OUProcess(1, sigma=0.01),
)

# Initialize states
node.init_all_states()

# Define simulation step
def step_run(i):
    with brainstate.environ.context(i=i, t=i * brainstate.environ.get_dt()):
        node.update()
        return node.x.value, node.y.value

# Run simulation (1000 ms)
indices = np.arange(10000)
x_trace, y_trace = brainstate.transform.for_loop(step_run, indices)

### Step 3: Visualize Single Node Dynamics

In [ ]:
t_ms = indices * brainstate.environ.get_dt()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Time series
axes[0].plot(t_ms, x_trace[:, 0], label='x(t)', linewidth=0.8)
axes[0].plot(t_ms, y_trace[:, 0], label='y(t)', linewidth=0.8, alpha=0.7)
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Activity')
axes[0].set_title('Single Hopf Oscillator with OU Noise')
axes[0].legend()
axes[0].set_xlim([0, 500])

# Phase portrait
axes[1].plot(x_trace[:, 0], y_trace[:, 0], 'k-', linewidth=0.3, alpha=0.5)
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_title('Phase Portrait (Limit Cycle)')
axes[1].set_aspect('equal')

plt.tight_layout()
plt.show()

### Step 4: Bifurcation Diagram

The bifurcation diagram shows how system behavior changes with parameter $a$:

- **a < 0**: Origin is a stable focus (trajectories spiral inward)
- **a = 0**: Hopf bifurcation occurs
- **a > 0**: Stable limit cycle emerges with radius $\sqrt{a/\beta}$

In [ ]:
# Create nodes with varying bifurcation parameter
a_vals = np.linspace(-2, 2, 50)

nodes = brainmass.HopfStep(
    a_vals.size,
    a=a_vals,             # Different a for each node
    w=0.2,
    beta=1.0,
    init_x=braintools.init.Uniform(0, 1),
    init_y=braintools.init.Uniform(0, 1),
)
brainstate.nn.init_all_states(nodes)

# Simulate
def step_run(i):
    with brainstate.environ.context(i=i, t=i * brainstate.environ.get_dt()):
        return nodes.update()

indices = np.arange(20000)
x_trace = brainstate.transform.for_loop(step_run, indices)[0]

# Extract max/min from last 5000 steps (after transient)
x_last = x_trace[-5000:]
max_x = x_last.max(axis=0)
min_x = x_last.min(axis=0)

In [ ]:
plt.figure(figsize=(8, 5))
plt.fill_between(a_vals, min_x, max_x, alpha=0.3, color='blue', label='Oscillation envelope')
plt.plot(a_vals, max_x, 'b-', lw=2, label='max(x)')
plt.plot(a_vals, min_x, 'b-', lw=2, label='min(x)')
plt.axvline(0.0, ls='--', color='red', label='Bifurcation (a=0)')
plt.axhline(0.0, ls=':', color='gray', alpha=0.5)

plt.xlabel('Bifurcation parameter (a)', fontsize=12)
plt.ylabel('Oscillation amplitude', fontsize=12)
plt.title('Hopf Bifurcation Diagram', fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Step 5: Brain Network with Diffusive Coupling

For $N$ coupled brain regions, we extend to a network with diffusive coupling:

$$
\begin{aligned}
\dot{x}_i &= (a - \beta r_i)x_i - \omega y_i + K \sum_j W_{ij}(x_j - x_i) + I_i^x(t) \\
\dot{y}_i &= (a - \beta r_i)y_i + \omega x_i + K \sum_j W_{ij}(y_j - y_i) + I_i^y(t)
\end{aligned}
$$

where $W$ is the structural connectivity matrix and $K$ is the global coupling strength.

In [ ]:
# Load Human Connectome Project data
import os.path
import kagglehub

path = kagglehub.dataset_download("oujago/hcp-gw-data-samples")
data = braintools.file.msgpack_load(os.path.join(path, "hcp-data-sample.msgpack"))

print(f"Connectome shape: {data['Cmat'].shape}")
print(f"Distance matrix shape: {data['Dmat'].shape}")

In [ ]:
class HopfNetwork(brainstate.nn.Module):
    """A network of coupled Hopf oscillators representing brain regions."""
    
    def __init__(self, signal_speed=2.0, coupling_strength=1.0):
        super().__init__()
        
        # Structural connectivity (weights)
        conn_weight = data['Cmat'].copy()
        np.fill_diagonal(conn_weight, 0)
        self.conn_weight = conn_weight
        
        # Axonal delays from distance matrix
        delay_time = data['Dmat'].copy() / signal_speed
        np.fill_diagonal(delay_time, 0)
        delay_time = delay_time * u.ms
        indices_ = np.expand_dims(np.arange(conn_weight.shape[1]), 0)
        indices_ = np.repeat(indices_, conn_weight.shape[0], axis=0)
        delay_index = (delay_time, indices_)
        
        # Create Hopf nodes
        n_node = conn_weight.shape[1]
        self.n_node = n_node
        self.node = brainmass.HopfStep(
            n_node,
            w=1.0,
            a=0.25,
            init_x=braintools.init.Uniform(0, 0.5),
            init_y=braintools.init.Uniform(0, 0.5),
            noise_x=brainmass.OUProcess(n_node, sigma=0.14, tau=5.0 * u.ms),
            noise_y=brainmass.OUProcess(n_node, sigma=0.14, tau=5.0 * u.ms),
        )
        
        # Diffusive coupling on x-component
        self.coupling_x = brainmass.DiffusiveCoupling(
            self.node.prefetch_delay('x', delay_index, init=braintools.init.Uniform(0, 0.05)),
            self.node.prefetch('x'),
            conn_weight,
            k=coupling_strength
        )
    
    def update(self):
        self.node.update(x_inp=self.coupling_x())
        return self.node.x.value, self.node.y.value
    
    def step_run(self, i):
        with brainstate.environ.context(i=i, t=i * brainstate.environ.get_dt()):
            return self.update()

In [ ]:
# Create and run network
dt = brainstate.environ.get_dt()
net = HopfNetwork(coupling_strength=1.0)
brainstate.nn.init_all_states(net)

# Simulate 500 ms
indices = np.arange(0, int(0.5 * u.second // dt))
exes_tuple = brainstate.transform.for_loop(net.step_run, indices)
exes_x = jax.block_until_ready(exes_tuple[0])
exes_y = jax.block_until_ready(exes_tuple[1])

print(f"Simulated {net.n_node} brain regions for {len(indices) * dt}")

### Step 6: Analyze Network Dynamics

In [ ]:
plt.rcParams['image.cmap'] = 'inferno'

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Structural connectivity
im0 = axes[0].imshow(net.conn_weight, aspect='auto')
axes[0].set_title('Structural Connectivity')
axes[0].set_xlabel('Region j')
axes[0].set_ylabel('Region i')
plt.colorbar(im0, ax=axes[0], shrink=0.8)

# Functional connectivity (from simulated data)
fc = braintools.metric.functional_connectivity(exes_x)
im1 = axes[1].imshow(fc, aspect='auto', vmin=-1, vmax=1, cmap='RdBu_r')
axes[1].set_title('Functional Connectivity')
axes[1].set_xlabel('Region j')
axes[1].set_ylabel('Region i')
plt.colorbar(im1, ax=axes[1], shrink=0.8)

# Time series of selected regions
t_ms = indices * dt
axes[2].plot(t_ms, exes_x[:, ::10], alpha=0.7, linewidth=0.5)
axes[2].set_xlabel('Time (ms)')
axes[2].set_ylabel('Activity (x)')
axes[2].set_title('Regional Time Series (every 10th region)')

plt.tight_layout()
plt.show()

## Exercises

### Exercise 1: Frequency Analysis

The oscillation frequency depends on both $\omega$ and the coupling. Modify the network to use different $\omega$ values and observe:
1. How does the dominant frequency change?
2. How does coupling affect frequency?

*Hint*: Use `np.fft.fft` to compute the power spectrum.

### Exercise 2: Coupling Strength

Explore different coupling strengths:
```python
for k in [0.1, 1.0, 5.0, 10.0]:
    net = HopfNetwork(coupling_strength=k)
    # Run and analyze...
```

Questions:
1. What happens at very weak coupling?
2. What happens at very strong coupling?
3. How does functional connectivity change with coupling strength?

### Exercise 3: Subcritical Regime

Set `a=-0.1` (below bifurcation) and observe how noise can induce oscillations even in the subcritical regime. This demonstrates **noise-induced oscillations**.

## Summary

In this tutorial, you learned:

1. **Hopf oscillator mathematics**: The normal form equations and parameter roles
2. **Bifurcation analysis**: How the system transitions from quiescent to oscillatory
3. **Network modeling**: Coupling multiple brain regions with structural connectivity
4. **Functional connectivity**: Computing correlations between simulated regional activities

### Key Insights

- The Hopf model is a **canonical oscillator** that captures essential features of neural rhythms
- **Diffusive coupling** naturally models how brain regions influence each other
- **Delays** from axonal propagation are crucial for realistic brain network dynamics

### Next Steps

- **Tutorial 011**: Stuart-Landau oscillator (alternative Hopf formulation)
- **Tutorial 030**: Kuramoto model for phase synchronization
- **Tutorial 080**: Parameter exploration and optimization

## References

1. Deco, G., Kringelbach, M. L., Jirsa, V. K., & Ritter, P. (2017). The dynamics of resting fluctuations in the brain: metastability and its dynamical cortical core. *Scientific Reports*, 7(1), 3095.

2. Kuznetsov, Y. A. (2013). *Elements of applied bifurcation theory* (Vol. 112). Springer Science & Business Media.

3. Deco, G., et al. (2009). Key role of coupling, delay, and noise in resting brain fluctuations. *PNAS*, 106(25), 10302-10307.